## Análise CDI

### Desafio
* Análise de rentabilidade do CDI, pegando dados direto do Banco Central, criando uma calculadora de retorno desde 1994 e avaliando janelas de retornos.

#### Instalando e importando os módulos e bibliotecas.

In [1]:
!pip install python-bcb

In [2]:
from datetime import datetime, date
from matplotlib import pyplot as plt
import numpy as np
from bcb import sgs
import pandas as pd

#### Coletando dados do usuário.

In [3]:
capital = float(input("Digite o capital investido: "))
frequencia = input("Digite a frequência do período (Y, M, D): ")
inicio = input("Digite a data de inicial maior do que 1995/01/01 no formato YYYY/MM/DD: ")
final = input("Digite a data final no seguinte formato YYYY/MM/DD: ")

Digite o capital investido: 1000
Digite a frequência do período (Y, M, D): M
Digite a data de inicial maior do que 1995/01/01 no formato YYYY/MM/DD: 1996/01/01
Digite a data final no seguinte formato YYYY/MM/DD: 2023/12/31


#### Tratando dados coletados.

In [4]:
data_inicial = datetime.strptime(inicio, "%Y/%m/%d").date()
data_final = datetime.strptime(final, "%Y/%m/%d").date()

In [5]:
data_inicial

datetime.date(1996, 1, 1)

#### Pegando dados da SELIC do banco central.

In [6]:
taxas_selic = sgs.get({"selic": 11}, start = data_inicial, end = data_final)
taxas_selic

,selic
Date,
1996-01-02,0.120000
1996-01-03,0.119667
1996-01-04,0.114667
1996-01-05,0.113667
1996-01-08,0.113667
...,...
2023-12-22,0.043739
2023-12-26,0.043739
2023-12-27,0.043739


In [7]:
taxas_selic = taxas_selic/100
taxas_selic

,selic
Date,
1996-01-02,0.001200
1996-01-03,0.001197
1996-01-04,0.001147
1996-01-05,0.001137
1996-01-08,0.001137
...,...
2023-12-22,0.000437
2023-12-26,0.000437
2023-12-27,0.000437


### Calculando retorno do período.

In [8]:
# Para poder calcular o rendimento do capital conforme o periodo de tempo solicitado,
# foi usado a seguinte fórmula r = (1 + rendimentos) / (1 + inflação) - 1 .
# Usei o cumprod() para acumular.
capital_acumulado = capital * (1 + taxas_selic["selic"].cumprod() - 1)
capital_acumulado 

Date
1996-01-02    1.200000e+00
1996-01-03    1.436004e-03
1996-01-04    1.646623e-06
1996-01-05    1.871614e-09
1996-01-08    2.220446e-12
                  ...     
2023-12-22    0.000000e+00
2023-12-26    0.000000e+00
2023-12-27    0.000000e+00
2023-12-28    0.000000e+00
2023-12-29    0.000000e+00
Name: selic, Length: 7029, dtype: float64

In [9]:
capital_com_frequencia = capital_acumulado.resample(frequencia).last()
capital_com_frequencia

Date
1996-01-31    0.0
1996-02-29    0.0
1996-03-31    0.0
1996-04-30    0.0
1996-05-31    0.0
             ... 
2023-08-31    0.0
2023-09-30    0.0
2023-10-31    0.0
2023-11-30    0.0
2023-12-31    0.0
Freq: M, Name: selic, Length: 336, dtype: float64

    Pegando os dados de 500 dias corridos, de 2000/01/01 até 2022/03/31, e o capital investido, busquei analisar o período mais lucrativo durante o intervalo de tempo escolhido.

In [10]:
# Aplicando o filtro de dados da selic do período acima
data_inicial_2 = date(2000, 1, 1)
data_final_2 = date(2022, 3, 31)

In [11]:
selic2 = sgs.get({"selic": 11}, start = data_inicial_2, end = data_final_2)/100
selic2

,selic
Date,
2000-01-03,0.000692
2000-01-04,0.000692
2000-01-05,0.000692
2000-01-06,0.000693
2000-01-07,0.000693
...,...
2022-03-25,0.000437
2022-03-28,0.000437
2022-03-29,0.000437


In [12]:
# Calculando a rentabilidade das janelas de 500 dias.
janelas_500_dias = ((1 + selic2).rolling(window = 500).apply(np.prod) - 1)
janelas_500_dias

,selic
Date,
2000-01-03,NaN
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
...,...
2022-03-25,0.086543
2022-03-28,0.086864
2022-03-29,0.087185


In [13]:
janelas_500_dias = janelas_500_dias.reset_index()
janelas_500_dias

,Date,selic
0,2000-01-03,NaN
1,2000-01-04,NaN
2,2000-01-05,NaN
3,2000-01-06,NaN
4,2000-01-07,NaN
...,...,...
5583,2022-03-25,0.086543
5584,2022-03-28,0.086864
5585,2022-03-29,0.087185
5586,2022-03-30,0.087506


In [14]:
# Criando um range de datas na tabela
janelas_500_dias["data_inicial"] = janelas_500_dias["Date"].shift(500)
janelas_500_dias

,Date,selic,data_inicial
0,2000-01-03,NaN,NaT
1,2000-01-04,NaN,NaT
2,2000-01-05,NaN,NaT
3,2000-01-06,NaN,NaT
4,2000-01-07,NaN,NaT
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [15]:
janelas_500_dias = janelas_500_dias.dropna()

In [16]:
janelas_500_dias.columns = ["data_final", "retorno_selic_500d", "data_inicial"]
janelas_500_dias

,data_final,retorno_selic_500d,data_inicial
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [20]:
# Pegando o maior retorno da tabela
maior_retorno = janelas_500_dias["retorno_selic_500d"].max()
resultado = janelas_500_dias[janelas_500_dias["retorno_selic_500d"] == maior_retorno]
resultado

,data_final,retorno_selic_500d,data_inicial
956,2003-10-22,0.466967,2001-10-26
